In [4]:
import sys
!{sys.executable} -m pip install -r requirements.txt
import pydicom
import matplotlib.pylab as plt
import matplotlib
import os
import numpy as np
from PIL import Image
import time
import skimage.io as io
from unet.model import *
from unet.data import *


You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


Configure Global Variables
----

In [5]:
window = 350 # Window/Level from Hoori et al ("HU-attention-window with a window/level of 350/40 HU")
level = 40
bitDepth = 8 # Must be 8 for uint8 PNGs. Explore floating point
TrainingDataRootDir = '../Training Data'
TestDataRootDir = '../Test Data'

Postprocess Test Data
----

In [2]:
CombinedTestInputImageDir = TestDataRootDir + '/AllImages'
CombinedTestInputMaskDir = TestDataRootDir + '/AllMasks'
CombinedTestOutputMaskDir = TestDataRootDir + '/AllOutputMasks'
CombinedTestOutputImageDir = TestDataRootDir + '/AllOutputImages'
os.makedirs(CombinedTestOutputMaskDir, exist_ok=True)
os.makedirs(CombinedTestOutputImageDir, exist_ok=True)

model = unet()
model.load_weights("unet_EAT_model_4_6_22.hdf")

mask_threshold = 0.2

currentImageIndex = 0
images = os.listdir(CombinedTestInputImageDir)
for image in images:
    test_image_path = CombinedTestInputImageDir + "/" + image
    print(test_image_path)
    inputImage = io.imread(test_image_path,as_gray = False)
    inputImage = inputImage / 255
    inputImage = trans.resize(inputImage,(256,256))
    inputImage = inputImage[:,:,0:3]
    inputImage = np.reshape(inputImage,(1,)+inputImage.shape)
    result = model.predict(inputImage)
    shape = result.shape
    mask_thresholded = result[0,:,:,0] > mask_threshold
    resultMaskNumVoxels = np.count_nonzero(mask_thresholded)
    print(resultMaskNumVoxels)

    test_mask_path = CombinedTestInputMaskDir + "/" + image
    inputMask = io.imread(test_path,as_gray = False)
    inputMask = inputMask / 255
    inputMask = trans.resize(inputMask,(256,256))
    inputMask = inputMask[:,:,0:3]
    inputMask = np.reshape(inputMask,(1,)+inputMask.shape)
    input_mask_thresholded = inputMask[0,:,:,0] > mask_threshold
    inputMaskNumVoxels = np.count_nonzero(input_mask_thresholded)
    # print(inputMaskNumVoxels)

    maskScaled = mask_thresholded * ((2**8)-1)
    # print(maskScaled.shape)
    maskRGB = np.zeros((shape[1],shape[2],3))
    maskRGB[:,:,0] = maskScaled
    maskRGB[:,:,1] = maskScaled
    maskRGB[:,:,2] = maskScaled
    matplotlib.image.imsave(os.path.join(CombinedTestOutputMaskDir,"%d.png"%currentImageIndex), maskRGB.astype(np.uint8))

    currentImageIndex = currentImageIndex + 1


NameError: name 'TestDataRootDir' is not defined

In [ ]:
## CA
## (0018, 0050) Slice Thickness                     DS: '2.5'
## (0028, 0010) Rows                                US: 512
## (0028, 0011) Columns                             US: 512
## (0028, 0030) Pixel Spacing                       DS: [0.8671875, 0.8671875]


## SA
## (0018, 0050) Slice Thickness                     DS: '2.5'
## (0028, 0010) Rows                                US: 512
## (0028, 0011) Columns                             US: 512
## (0028, 0030) Pixel Spacing                       DS: [0.78125, 0.78125]


Apply AI Fat Masks and Isolate Fat by HU
----

In [18]:
CombinedTestImageDir = TestDataRootDir + '/AllImages'
CombinedTestMaskDir = TestDataRootDir + '/AllMasks'
CombinedTestOutputMaskDir = TestDataRootDir + '/AllOutputMasks'
CombinedTestOutputImageDir = TestDataRootDir + '/AllOutputImages'
os.makedirs(CombinedTestOutputImageDir, exist_ok=True)

# fat
window = 80 
level = -110

currentImageIndex = 0
subjects = os.listdir(TestDataRootDir)
for subject in subjects:
    if not (subject == 'AllImages' or subject == 'AllMasks' or subject == 'AllOutputImages' or subject == 'AllOutputMasks'):
        currentSubjectDir = TestDataRootDir + "/" + subject;
        print(currentSubjectDir)

        volume_dicoms = os.listdir(currentSubjectDir + '/Dicom/Vol/')
        dataset_vol = [None] * (len(volume_dicoms))
        for dicom in volume_dicoms:
            slice_number = int(dicom.split('Slice')[1].split('.dcm')[0])
            dataset_vol[slice_number-1] = pydicom.dcmread(currentSubjectDir+'/Dicom/Vol/'+dicom)

        for i in range(1, len(dataset_vol)-1):
            sliceRGB = np.zeros((dataset_vol[i].pixel_array.shape[0],dataset_vol[i].pixel_array.shape[1],3))
            test_mask_result_path = os.path.join(CombinedTestOutputMaskDir,"%d.png"%currentImageIndex)
            resultMaskIn = io.imread(test_mask_result_path,as_gray = False)
            resultMaskScaled = resultMaskIn[:,:,1] / 255
            resultMaskResized = trans.resize(resultMaskScaled,(512,512))
            resultMask = resultMaskResized > 0.5
            maskedSlice = dataset_vol[i].pixel_array * resultMask
            maskedSlice = np.clip((maskedSlice - (level-window/2)) / window, 0,1) # Apply fat window/level and clip to 0->1 range
            for x in range(0,512):
                for y in range(0,512):
                    maskedSlice[x,y] = (maskedSlice[x,y]>0) and (maskedSlice[x,y]<1)
            maskedSlice = maskedSlice * ((2**bitDepth)-1) # Scale to bitdepth
            maskedSlice = np.floor(maskedSlice) # Floor to integer
            sliceRGB[:,:,0] = maskedSlice
            sliceRGB[:,:,1] = maskedSlice
            sliceRGB[:,:,2] = maskedSlice
            matplotlib.image.imsave(os.path.join(CombinedTestOutputImageDir,"%d.png"%currentImageIndex), sliceRGB.astype(np.uint8))
            currentImageIndex = currentImageIndex + 1

../Test Data/SA218
../Test Data/SA221
../Test Data/SA219
../Test Data/SA222


Apply Original Fat Masks and Isolate Fat by HU
----

In [6]:
CombinedTestImageDir = TestDataRootDir + '/AllImages'
CombinedTestMaskDir = TestDataRootDir + '/AllMasks'
CombinedTestOutputMaskDir = TestDataRootDir + '/AllOutputMasks'
CombinedTestOutputImageDir = TestDataRootDir + '/AllOutputImages'
CombinedTestReferenceImageDir = TestDataRootDir + '/AllReferenceImages'
CombinedTestOutputCombinedImageDir = TestDataRootDir + '/AllCombinedResultImages'
os.makedirs(CombinedTestOutputImageDir, exist_ok=True)
os.makedirs(CombinedTestReferenceImageDir, exist_ok=True)
os.makedirs(CombinedTestOutputCombinedImageDir, exist_ok=True)


window = 350 # Window/Level from Hoori et al ("HU-attention-window with a window/level of 350/40 HU")
level = 40
# fat
fatWindow = 80 
fatLevel = -110

currentImageIndex = 0
subjects = os.listdir(TestDataRootDir)
subjectVoxelCountsReference = []
subjectVoxelCountsResults = []

for subject in subjects:
    if not (subject == 'AllImages' or subject == 'AllMasks' or subject == 'AllOutputImages' or subject == 'AllOutputMasks' or subject == 'AllReferenceImages' or subject == 'AllCombinedResultImages'):
        currentSubjectDir = TestDataRootDir + "/" + subject
        print(currentSubjectDir)
        subjectVoxelCountReference = 0
        subjectVoxelCountResults = 0

        volume_dicoms = os.listdir(currentSubjectDir + '/Dicom/Vol/')
        dataset_vol = [None] * (len(volume_dicoms))
        for dicom in volume_dicoms:
            slice_number = int(dicom.split('Slice')[1].split('.dcm')[0])
            dataset_vol[slice_number-1] = pydicom.dcmread(currentSubjectDir+'/Dicom/Vol/'+dicom)

        for i in range(1, len(dataset_vol)-1):
            sliceRGB = np.zeros((dataset_vol[i].pixel_array.shape[0],dataset_vol[i].pixel_array.shape[1],3))
            originalInput = dataset_vol[i].pixel_array
            windowLeveledOriginalInput = np.clip((originalInput - (level-window/2)) / window, 0,1) # Apply fat window/level and clip to 0->1 range
            windowLeveledOriginalInput = windowLeveledOriginalInput * ((2**bitDepth)-1) # Scale to bitdepth
            windowLeveledOriginalInput = np.floor(windowLeveledOriginalInput) # Floor to integer

            test_mask_input_path = os.path.join(CombinedTestMaskDir,"%d.png"%currentImageIndex)
            MaskIn_input = io.imread(test_mask_input_path,as_gray = False)
            MaskScaled_input = MaskIn_input[:,:,1] / 255
            MaskResized_input = trans.resize(MaskScaled_input,(512,512))
            Mask_input = MaskResized_input > 0.5
            maskedSlice_input = originalInput * Mask_input
            maskedSlice_input = np.clip((maskedSlice_input - (fatLevel-fatWindow/2)) / fatWindow, 0,1) # Apply fat window/level and clip to 0->1 range


            test_mask_result_path = os.path.join(CombinedTestOutputMaskDir,"%d.png"%currentImageIndex)
            MaskIn_result = io.imread(test_mask_result_path,as_gray = False)
            MaskScaled_result = MaskIn_result[:,:,1] / 255
            MaskResized_result = trans.resize(MaskScaled_result,(512,512))
            Mask_result = MaskResized_result > 0.5
            maskedSlice_result = originalInput * Mask_result
            maskedSlice_result = np.clip((maskedSlice_result - (fatLevel-fatWindow/2)) / fatWindow, 0,1) # Apply fat window/level and clip to 0->1 range

            for x in range(0,512):
                for y in range(0,512):
                    maskedSlice_input[x,y] = (maskedSlice_input[x,y]>0) and (maskedSlice_input[x,y]<1)
                    maskedSlice_result[x,y] = (maskedSlice_result[x,y]>0) and (maskedSlice_result[x,y]<1)

            subjectVoxelCountReference = subjectVoxelCountReference + np.count_nonzero(maskedSlice_input)
            maskedSlice_input_scaled = maskedSlice_input * ((2**bitDepth)-1) # Scale to bitdepth
            maskedSlice_input_scaled = np.floor(maskedSlice_input_scaled) # Floor to integer

            subjectVoxelCountResults = subjectVoxelCountResults + np.count_nonzero(maskedSlice_result)
            maskedSlice_result_scaled = maskedSlice_result * ((2**bitDepth)-1) # Scale to bitdepth
            maskedSlice_result_scaled = np.floor(maskedSlice_result_scaled) # Floor to integer

            # Save original fat mask
            sliceRGB[:,:,0] = (windowLeveledOriginalInput * 0.5) + (0.5 * maskedSlice_input_scaled)
            sliceRGB[:,:,1] = (windowLeveledOriginalInput * 0.5) 
            sliceRGB[:,:,2] = (windowLeveledOriginalInput * 0.5) 
            matplotlib.image.imsave(os.path.join(CombinedTestReferenceImageDir,"%d.png"%currentImageIndex), sliceRGB.astype(np.uint8))

            # Save AI fat mask
            sliceRGB[:,:,0] = (windowLeveledOriginalInput * 0.5) 
            sliceRGB[:,:,1] = (windowLeveledOriginalInput * 0.5) 
            sliceRGB[:,:,2] = (windowLeveledOriginalInput * 0.5) + (0.5 * maskedSlice_result_scaled)
            matplotlib.image.imsave(os.path.join(CombinedTestOutputImageDir,"%d.png"%currentImageIndex), sliceRGB.astype(np.uint8))

            # Save Combined fat mask
            sliceRGB[:,:,0] = (windowLeveledOriginalInput * 0.5) + (0.5 * maskedSlice_input_scaled)
            sliceRGB[:,:,1] = (windowLeveledOriginalInput * 0.5) 
            sliceRGB[:,:,2] = (windowLeveledOriginalInput * 0.5) + (0.5 * maskedSlice_result_scaled)
            matplotlib.image.imsave(os.path.join(CombinedTestOutputCombinedImageDir,"%d.png"%currentImageIndex), sliceRGB.astype(np.uint8))

            currentImageIndex = currentImageIndex + 1
        subjectVoxelCountsReference.append(subjectVoxelCountReference)
        subjectVoxelCountsResults.append(subjectVoxelCountResults)


../Test Data/SA218
../Test Data/SA221
../Test Data/SA219
../Test Data/SA222


In [7]:
subjectVoxelCountsResults[0]

114303

In [8]:
subjectVoxelCountsReference[0]

107373

In [12]:
totalEATVolumeReference = []
totalEATVolumeResult = []
EATVolumeError = []
EATPercentError = []
for subjectNumber in range(0,len(subjectVoxelCountsResults)):
    totalEATVolumeReference.append(subjectVoxelCountsReference[subjectNumber]*2.5*0.8671875*0.8671875)
    totalEATVolumeResult.append(subjectVoxelCountsResults[subjectNumber]*2.5*0.8671875*0.8671875)
    EATVolumeError.append(np.abs(totalEATVolumeReference[subjectNumber] - totalEATVolumeResult[subjectNumber]))
    EATPercentError.append(EATVolumeError[subjectNumber] / totalEATVolumeReference[subjectNumber])

IndexError: list assignment index out of range